# 🧬 Training pipeline: Training ranking model </span>

In this notebook, you will train a ranking model using gradient boosted trees. 

In [3]:
%load_ext autoreload
%autoreload 2

import os
import warnings
import polars as pl

warnings.filterwarnings("ignore")

from recsys.config import settings
from recsys.data.preprocessing.splitting import train_test_split
from recsys.gcp.vertex_ai.serving.ranking import GCPRankingModel
from recsys.core.models.two_tower.ranking import (
    RankingModelFactory,
    RankingModelTrainer,
)
from recsys.gcp.feature_store import client as fs_client
from recsys.gcp.bigquery import client as bq_client

## 💿 Create training dataset

In [ ]:
path = os.getcwd()[:-9]
fullpath = os.path.join(path, 'data/preprocessed')

In [5]:
trans_df = pl.read_csv(f'{fullpath}/transactions.csv')
articles_df = pl.read_parquet(f'{fullpath}/articles.parquet')
customers_df = pl.read_csv(f'{fullpath}/customers.csv')
rankings_df = pl.read_csv(f'{fullpath}/ranking.csv')

In [6]:
rankings_df = rankings_df.join(
    trans_df.select(["customer_id", "month_sin", "month_cos"]),
    on="customer_id",
    how="left",
)

In [7]:
articles_df = articles_df.with_columns(pl.col('article_id').cast(pl.Int64))

rankings_df = rankings_df.join(
    articles_df.select(["article_id", "colour_group_name"]),
    on="article_id",
    how="left",
)

In [8]:
rankings_df = rankings_df.drop("customer_id", "article_id")

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    df=rankings_df,
    test_size=settings.RANKING_DATASET_VALIDATION_SPLIT_SIZE,
)

In [ ]:
X_train.columns

In [ ]:
y_train.head(3)

# Training the ranking model

Let's train the ranking model:

In [14]:
model = RankingModelFactory.build()

In [ ]:
trainer = RankingModelTrainer(
    model=model, train_dataset=(X_train, y_train), eval_dataset=(X_val, y_val)
)

In [ ]:
trainer.fit()

In [ ]:
optimal_threshold, best_metrics = trainer.find_optimal_threshold(step=0.05)

## Evaluating the ranking model

Next, you'll evaluate how well the model performs on the validation data using metrics for classification such as precision, recall and f1-score:

In [ ]:
metrics = trainer.evaluate(log=True, threshold=optimal_threshold)

In [ ]:
metrics

It can be seen that the model has a low F1-score on the positive class (higher is better). The performance could potentially be improved by adding more features to the dataset, e.g. image embeddings.

Let's see which features your model considers important.

In [ ]:
trainer.get_feature_importance()

# 🗄️ Save model to local

In [23]:
ranking_model_gcp = GCPRankingModel(model=model)

In [ ]:
ranking_model_gcp.save_to_local('ranking_model/ranking')